In [94]:
from nnAudio import Spectrogram
from scipy.io import wavfile
import torch
import soundfile as sf
import numpy as np

In [95]:
import scipy.io.wavfile as wav

rate,sig = wav.read('./BAC009S0764W0124.wav')
sr, song = wavfile.read('./BAC009S0764W0124.wav') # Loading your audio
sample, sr = sf.read('./BAC009S0764W0124.wav', dtype='int16')
print(sig)
print(song)
print(sample)

[43 75 69 ...  7  6  3]
[43 75 69 ...  7  6  3]
[43 75 69 ...  7  6  3]


In [96]:
sr, song = wavfile.read('./BAC009S0764W0124.wav') # Loading your audio
print(sr)
print(song)
print(song.shape)
print(song.dtype)
x = song
x = torch.tensor(x).float() # casting the array into a PyTorch Tensor

spec_layer = Spectrogram.STFT(n_fft=2048, freq_bins=None, hop_length=512,
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect',
                              fmin=50,fmax=8000, sr=sr) # Initializing the model

spec = spec_layer(x) # Feed-forward your waveform to get the spectrogram
print(spec)

16000
[43 75 69 ...  7  6  3]
(83792,)
int16
sampling rate = 16000. Please make sure the sampling rate is correct in order toget a valid freq range
STFT kernels created, time used = 0.2733 seconds
tensor([[[[-4.0940e+03,  1.2600e+04],
          [ 8.5108e+03, -5.4930e+03],
          [-3.3631e+03, -1.7904e+03],
          ...,
          [ 8.2279e+03, -9.3340e+03],
          [-3.1990e+03,  2.0969e+03],
          [-1.2669e+03,  4.4488e+03]],

         [[ 3.4886e+03, -9.9620e+03],
          [-4.5364e+03,  4.1907e+02],
          [ 2.5074e+03,  7.1339e+03],
          ...,
          [-5.4819e+03,  3.9258e+01],
          [ 4.7221e+03,  6.5887e+01],
          [ 9.6492e+02, -3.4386e+03]],

         [[-3.4947e+03,  9.2981e+03],
          [-7.5164e+03,  8.1856e+02],
          [-5.3766e+03, -9.0889e+03],
          ...,
          [ 1.4317e+03,  5.7447e+03],
          [-3.1178e+03,  3.0740e+03],
          [-3.4351e+03,  5.6900e+02]],

         ...,

         [[ 6.7112e+01, -4.5737e+00],
          [-9.6

In [97]:
wav, sr = sf.read('./BAC009S0764W0124.wav', dtype='int16')
print(sr)
print(wav)
print(wav.shape)
print(wav.dtype)
print(np.allclose(wav, song))

x = wav
x = torch.tensor(x).float() # casting the array into a PyTorch Tensor

spec_layer = Spectrogram.STFT(n_fft=2048, freq_bins=None, hop_length=512,
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect',
                              fmin=50,fmax=8000, sr=sr) # Initializing the model

wav_spec = spec_layer(x) # Feed-forward your waveform to get the spectrogram
print(wav_spec.shape)
print(wav_spec)
print(np.allclose(wav_spec, spec))

16000
[43 75 69 ...  7  6  3]
(83792,)
int16
True
sampling rate = 16000. Please make sure the sampling rate is correct in order toget a valid freq range
STFT kernels created, time used = 0.2001 seconds
torch.Size([1, 1025, 164, 2])
tensor([[[[-4.0940e+03,  1.2600e+04],
          [ 8.5108e+03, -5.4930e+03],
          [-3.3631e+03, -1.7904e+03],
          ...,
          [ 8.2279e+03, -9.3340e+03],
          [-3.1990e+03,  2.0969e+03],
          [-1.2669e+03,  4.4488e+03]],

         [[ 3.4886e+03, -9.9620e+03],
          [-4.5364e+03,  4.1907e+02],
          [ 2.5074e+03,  7.1339e+03],
          ...,
          [-5.4819e+03,  3.9258e+01],
          [ 4.7221e+03,  6.5887e+01],
          [ 9.6492e+02, -3.4386e+03]],

         [[-3.4947e+03,  9.2981e+03],
          [-7.5164e+03,  8.1856e+02],
          [-5.3766e+03, -9.0889e+03],
          ...,
          [ 1.4317e+03,  5.7447e+03],
          [-3.1178e+03,  3.0740e+03],
          [-3.4351e+03,  5.6900e+02]],

         ...,

         [[ 6.7112

In [98]:
import decimal

import numpy
import math
import logging
def round_half_up(number):
    return int(decimal.Decimal(number).quantize(decimal.Decimal('1'), rounding=decimal.ROUND_HALF_UP))


def rolling_window(a, window, step=1):
    # http://ellisvalentiner.com/post/2017-03-21-np-strides-trick
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return numpy.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)[::step]


def framesig(sig, frame_len, frame_step, dither=1.0, preemph=0.97, remove_dc_offset=True, wintype='hamming', stride_trick=True):
    """Frame a signal into overlapping frames.

    :param sig: the audio signal to frame.
    :param frame_len: length of each frame measured in samples.
    :param frame_step: number of samples after the start of the previous frame that the next frame should begin.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied.
    :param stride_trick: use stride trick to compute the rolling window and window multiplication faster
    :returns: an array of frames. Size is NUMFRAMES by frame_len.
    """
    slen = len(sig)
    frame_len = int(round_half_up(frame_len))
    frame_step = int(round_half_up(frame_step))
    if slen <= frame_len:
        numframes = 1
    else:
        numframes = 1 + (( slen - frame_len) // frame_step)

    # check kaldi/src/feat/feature-window.h
    padsignal = sig[:(numframes-1)*frame_step+frame_len]
    if wintype is 'povey':
        win = numpy.empty(frame_len)
        for i in range(frame_len):
            win[i] = (0.5-0.5*numpy.cos(2*numpy.pi/(frame_len-1)*i))**0.85     
    else: # the hamming window
        win = numpy.hamming(frame_len)
        
    if stride_trick:
        frames = rolling_window(padsignal, window=frame_len, step=frame_step)
    else:
        indices = numpy.tile(numpy.arange(0, frame_len), (numframes, 1)) + numpy.tile(
            numpy.arange(0, numframes * frame_step, frame_step), (frame_len, 1)).T
        indices = numpy.array(indices, dtype=numpy.int32)
        frames = padsignal[indices]
        win = numpy.tile(win, (numframes, 1))
        
    frames = frames.astype(numpy.float32)
    raw_frames = numpy.zeros(frames.shape)
    for frm in range(frames.shape[0]):
        raw_frames[frm,:] = frames[frm,:]
        frames[frm,:] = do_dither(frames[frm,:], dither)        # dither
        frames[frm,:] = do_remove_dc_offset(frames[frm,:])      # remove dc offset
        # raw_frames[frm,:] = frames[frm,:]
        frames[frm,:] = do_preemphasis(frames[frm,:], preemph)    # preemphasize

    return frames * win, raw_frames


def magspec(frames, NFFT):
    """Compute the magnitude spectrum of each frame in frames. If frames is an NxD matrix, output will be Nx(NFFT/2+1).

    :param frames: the array of frames. Each row is a frame.
    :param NFFT: the FFT length to use. If NFFT > frame_len, the frames are zero-padded.
    :returns: If frames is an NxD matrix, output will be Nx(NFFT/2+1). Each row will be the magnitude spectrum of the corresponding frame.
    """
    if numpy.shape(frames)[1] > NFFT:
        logging.warn(
            'frame length (%d) is greater than FFT size (%d), frame will be truncated. Increase NFFT to avoid.',
            numpy.shape(frames)[1], NFFT)
    complex_spec = numpy.fft.rfft(frames, NFFT)
    return numpy.absolute(complex_spec)


def powspec(frames, NFFT):
    """Compute the power spectrum of each frame in frames. If frames is an NxD matrix, output will be Nx(NFFT/2+1).

    :param frames: the array of frames. Each row is a frame.
    :param NFFT: the FFT length to use. If NFFT > frame_len, the frames are zero-padded.
    :returns: If frames is an NxD matrix, output will be Nx(NFFT/2+1). Each row will be the power spectrum of the corresponding frame.
    """
    return numpy.square(magspec(frames, NFFT))


def do_dither(signal, dither_value=1.0):
    signal += numpy.random.normal(size=signal.shape) * dither_value
    return signal
    
def do_remove_dc_offset(signal):
    signal -= numpy.mean(signal)
    return signal

def do_preemphasis(signal, coeff=0.97):
    """perform preemphasis on the input signal.

    :param signal: The signal to filter.
    :param coeff: The preemphasis coefficient. 0 is no filter, default is 0.95.
    :returns: the filtered signal.
    """
    return numpy.append((1-coeff)*signal[0], signal[1:] - coeff * signal[:-1])

In [99]:
def fbank(signal,samplerate=16000,winlen=0.025,winstep=0.01,
          nfilt=40,nfft=512,lowfreq=0,highfreq=None,dither=1.0,remove_dc_offset=True, preemph=0.97, 
          wintype='hamming'):
    highfreq= highfreq or samplerate/2
    frames, raw_frames = framesig(signal, winlen*samplerate, winstep*samplerate, dither, preemph, remove_dc_offset, wintype)
    spec = magspec(frames, nfft) # nearly the same until this part
    rspec = magspec(raw_frames, nfft)
    return spec, rspec



def frames(signal,samplerate=16000,winlen=0.025,winstep=0.01,
          nfilt=40,nfft=512,lowfreq=0,highfreq=None,dither=1.0,remove_dc_offset=True, preemph=0.97, 
          wintype='hamming'):
    highfreq= highfreq or samplerate/2
    frames, raw_frames = framesig(signal, winlen*samplerate, winstep*samplerate, dither, preemph, remove_dc_offset, wintype)
    return raw_frames

In [100]:

import torch
import torch.nn as nn
from torch.nn.functional import conv1d, conv2d, fold
import scipy # used only in CFP

import numpy as np
from time import time

def pad_center(data, size, axis=-1, **kwargs):

    kwargs.setdefault('mode', 'constant')

    n = data.shape[axis]

    lpad = int((size - n) // 2)

    lengths = [(0, 0)] * data.ndim
    lengths[axis] = (lpad, int(size - n - lpad))

    if lpad < 0:
        raise ParameterError(('Target size ({:d}) must be '
                              'at least input size ({:d})').format(size, n))

    return np.pad(data, lengths, **kwargs)



sz_float = 4    # size of a float
epsilon = 10e-8 # fudge factor for normalization

def create_fourier_kernels(n_fft, win_length=None, freq_bins=None, fmin=50,fmax=6000, sr=44100,
                           freq_scale='linear', window='hann', verbose=True):

    if freq_bins==None: freq_bins = n_fft//2+1
    if win_length==None: win_length = n_fft

    s = np.arange(0, n_fft, 1.)
    wsin = np.empty((freq_bins,1,n_fft))
    wcos = np.empty((freq_bins,1,n_fft))
    start_freq = fmin
    end_freq = fmax
    bins2freq = []
    binslist = []

    # num_cycles = start_freq*d/44000.
    # scaling_ind = np.log(end_freq/start_freq)/k

    # Choosing window shape

    #window_mask = get_window(window, int(win_length), fftbins=True)
    window_mask = np.hamming(int(win_length))
    window_mask = pad_center(window_mask, n_fft)

    if freq_scale == 'linear':
        if verbose==True:
            print(f"sampling rate = {sr}. Please make sure the sampling rate is correct in order to"
                  f"get a valid freq range")
            
        start_bin = start_freq*n_fft/sr
        scaling_ind = (end_freq-start_freq)*(n_fft/sr)/freq_bins

        for k in range(freq_bins): # Only half of the bins contain useful info
            # print("linear freq = {}".format((k*scaling_ind+start_bin)*sr/n_fft))
            bins2freq.append((k*scaling_ind+start_bin)*sr/n_fft)
            binslist.append((k*scaling_ind+start_bin))
            wsin[k,0,:] = np.sin(2*np.pi*(k*scaling_ind+start_bin)*s/n_fft)
            wcos[k,0,:] = np.cos(2*np.pi*(k*scaling_ind+start_bin)*s/n_fft)

    elif freq_scale == 'log':
        if verbose==True:
            print(f"sampling rate = {sr}. Please make sure the sampling rate is correct in order to"
                  f"get a valid freq range")
        start_bin = start_freq*n_fft/sr
        scaling_ind = np.log(end_freq/start_freq)/freq_bins

        for k in range(freq_bins): # Only half of the bins contain useful info
            # print("log freq = {}".format(np.exp(k*scaling_ind)*start_bin*sr/n_fft))
            bins2freq.append(np.exp(k*scaling_ind)*start_bin*sr/n_fft)
            binslist.append((np.exp(k*scaling_ind)*start_bin))
            wsin[k,0,:] = np.sin(2*np.pi*(np.exp(k*scaling_ind)*start_bin)*s/n_fft)
            wcos[k,0,:] = np.cos(2*np.pi*(np.exp(k*scaling_ind)*start_bin)*s/n_fft)

    elif freq_scale == 'no':
        for k in range(freq_bins): # Only half of the bins contain useful info
            bins2freq.append(k*sr/n_fft)
            binslist.append(k)
            wsin[k,0,:] = np.sin(2*np.pi*k*s/n_fft)
            wcos[k,0,:] = np.cos(2*np.pi*k*s/n_fft)
    else:
        print("Please select the correct frequency scale, 'linear' or 'log'")
    return wsin.astype(np.float32),wcos.astype(np.float32), bins2freq, binslist, window_mask.astype(np.float32)



def broadcast_dim(x):
    """
    Auto broadcast input so that it can fits into a Conv1d
    """

    if x.dim() == 2:
        x = x[:, None, :]
    elif x.dim() == 1:
        # If nn.DataParallel is used, this broadcast doesn't work
        x = x[None, None, :]
    elif x.dim() == 3:
        pass
    else:
        raise ValueError("Only support input with shape = (batch, len) or shape = (len)")
    return x



### --------------------------- Spectrogram Classes ---------------------------###
class STFT(torch.nn.Module):

    def __init__(self, n_fft=2048, win_length=None, freq_bins=None, hop_length=None, window='hann',
                freq_scale='no', center=True, pad_mode='reflect', iSTFT=False,
                fmin=50, fmax=6000, sr=22050, trainable=False,
                output_format="Complex", verbose=True):

        super().__init__()

        # Trying to make the default setting same as librosa
        if win_length==None: win_length = n_fft
        if hop_length==None: hop_length = int(win_length // 4)

        self.output_format = output_format
        self.trainable = trainable
        self.stride = hop_length
        self.center = center
        self.pad_mode = pad_mode
        self.n_fft = n_fft
        self.freq_bins = freq_bins
        self.trainable = trainable
        self.pad_amount = self.n_fft // 2
        self.window = window
        self.win_length = win_length
        self.iSTFT = iSTFT
        self.trainable = trainable
        start = time()



        # Create filter windows for stft
        kernel_sin, kernel_cos, self.bins2freq, self.bin_list, window_mask = create_fourier_kernels(n_fft,
                                                                           win_length=win_length,
                                                                           freq_bins=freq_bins,
                                                                           window=window,
                                                                           freq_scale=freq_scale,
                                                                           fmin=fmin,
                                                                           fmax=fmax,
                                                                           sr=sr,
                                                                           verbose=verbose)


        kernel_sin = torch.tensor(kernel_sin, dtype=torch.float)
        kernel_cos = torch.tensor(kernel_cos, dtype=torch.float)
        
        # In this way, the inverse kernel and the forward kernel do not share the same memory...
        kernel_sin_inv = torch.cat((kernel_sin, -kernel_sin[1:-1].flip(0)), 0)
        kernel_cos_inv = torch.cat((kernel_cos, kernel_cos[1:-1].flip(0)), 0)
        
        if iSTFT:
            self.register_buffer('kernel_sin_inv', kernel_sin_inv.unsqueeze(-1))
            self.register_buffer('kernel_cos_inv', kernel_cos_inv.unsqueeze(-1))

        # Applying window functions to the Fourier kernels
        if window:
            window_mask = torch.tensor(window_mask)
            wsin = kernel_sin * window_mask
            wcos = kernel_cos * window_mask
        else:
            wsin = kernel_sin
            wcos = kernel_cos
        
        if self.trainable==False:
            self.register_buffer('wsin', wsin)
            self.register_buffer('wcos', wcos)            
        
        if self.trainable==True:
            wsin = torch.nn.Parameter(wsin, requires_grad=self.trainable)
            wcos = torch.nn.Parameter(wcos, requires_grad=self.trainable)  
            self.register_parameter('wsin', wsin)
            self.register_parameter('wcos', wcos)         
        
        # Prepare the shape of window mask so that it can be used later in inverse
        # self.register_buffer('window_mask', window_mask.unsqueeze(0).unsqueeze(-1))
        
        if verbose==True:
            print("STFT kernels created, time used = {:.4f} seconds".format(time()-start))
        else:
            pass

    def forward(self, x, output_format=None):
        """
        Convert a batch of waveforms to spectrograms.
        
        Parameters
        ----------
        x : torch tensor
            Input signal should be in either of the following shapes.\n
            1. ``(len_audio)``\n
            2. ``(num_audio, len_audio)``\n
            3. ``(num_audio, 1, len_audio)``
            It will be automatically broadcast to the right shape
        
        output_format : str
            Control the type of spectrogram to be return. Can be either ``Magnitude`` or ``Complex`` or ``Phase``.
            Default value is ``Complex``.  
            
        """
        output_format = output_format or self.output_format
        self.num_samples = x.shape[-1]
        
        x = broadcast_dim(x)
        if self.center:
            if self.pad_mode == 'constant':
                padding = nn.ConstantPad1d(self.pad_amount, 0)

            elif self.pad_mode == 'reflect':
                if self.num_samples < self.pad_amount:
                    raise AssertionError("Signal length shorter than reflect padding length (n_fft // 2).")
                padding = nn.ReflectionPad1d(self.pad_amount)

            x = padding(x)
        spec_imag = conv1d(x, self.wsin, stride=self.stride)
        spec_real = conv1d(x, self.wcos, stride=self.stride)  # Doing STFT by using conv1d

        # remove redundant parts
        spec_real = spec_real[:, :self.freq_bins, :]
        spec_imag = spec_imag[:, :self.freq_bins, :]

        if output_format=='Magnitude':
            spec = spec_real.pow(2) + spec_imag.pow(2)
            if self.trainable==True:
                return torch.sqrt(spec+1e-8)  # prevent Nan gradient when sqrt(0) due to output=0
            else:
                return torch.sqrt(spec)

        elif output_format=='Complex':
            return torch.stack((spec_real,-spec_imag), -1)  # Remember the minus sign for imaginary part

        elif output_format=='Phase':
            return torch.atan2(-spec_imag+0.0,spec_real)  # +0.0 removes -0.0 elements, which leads to error in calculating phase

    def inverse(self, X, onesided=True, length=None, refresh_win=True):
        """
        This function is same as the :func:`~nnAudio.Spectrogram.iSTFT` class, 
        which is to convert spectrograms back to waveforms. 
        It only works for the complex value spectrograms. If you have the magnitude spectrograms,
        please use :func:`~nnAudio.Spectrogram.Griffin_Lim`. 
        
        Parameters
        ----------
        onesided : bool
            If your spectrograms only have ``n_fft//2+1`` frequency bins, please use ``onesided=True``,
            else use ``onesided=False``

        length : int
            To make sure the inverse STFT has the same output length of the original waveform, please
            set `length` as your intended waveform length. By default, ``length=None``,
            which will remove ``n_fft//2`` samples from the start and the end of the output.
            
        refresh_win : bool
            Recalculating the window sum square. If you have an input with fixed number of timesteps,
            you can increase the speed by setting ``refresh_win=False``. Else please keep ``refresh_win=True``
           
           
        """
        if (hasattr(self, 'kernel_sin_inv') != True) or (hasattr(self, 'kernel_cos_inv') != True):
            raise NameError("Please activate the iSTFT module by setting `iSTFT=True` if you want to use `inverse`")      
        
        assert X.dim()==4 , "Inverse iSTFT only works for complex number," \
                            "make sure our tensor is in the shape of (batch, freq_bins, timesteps, 2)."\
                            "\nIf you have a magnitude spectrogram, please consider using Griffin-Lim."
        if onesided:
            X = extend_fbins(X) # extend freq

    
        X_real, X_imag = X[:, :, :, 0], X[:, :, :, 1]

        # broadcast dimensions to support 2D convolution
        X_real_bc = X_real.unsqueeze(1)
        X_imag_bc = X_imag.unsqueeze(1)
        a1 = conv2d(X_real_bc, self.kernel_cos_inv, stride=(1,1))
        b2 = conv2d(X_imag_bc, self.kernel_sin_inv, stride=(1,1))
       
        # compute real and imag part. signal lies in the real part
        real = a1 - b2
        real = real.squeeze(-2)*self.window_mask

        # Normalize the amplitude with n_fft
        real /= (self.n_fft)

        # Overlap and Add algorithm to connect all the frames
        real = overlap_add(real, self.stride)
    
        # Prepare the window sumsqure for division
        # Only need to create this window once to save time
        # Unless the input spectrograms have different time steps
        if hasattr(self, 'w_sum')==False or refresh_win==True:
            self.w_sum = torch_window_sumsquare(self.window_mask.flatten(), X.shape[2], self.stride, self.n_fft).flatten()
            self.nonzero_indices = (self.w_sum>1e-10)    
        else:
            pass
        real[:, self.nonzero_indices] = real[:,self.nonzero_indices].div(self.w_sum[self.nonzero_indices])
        # Remove padding
        if length is None:       
            if self.center:
                real = real[:, self.pad_amount:-self.pad_amount]

        else:
            if self.center:
                real = real[:, self.pad_amount:self.pad_amount + length]    
            else:
                real = real[:, :length] 
            
        return real
    
    def extra_repr(self) -> str:
        return 'n_fft={}, Fourier Kernel size={}, iSTFT={}, trainable={}'.format(
            self.n_fft, (*self.wsin.shape,), self.iSTFT, self.trainable
        )    

In [128]:
wav, sr = sf.read('./BAC009S0764W0124.wav', dtype='int16')
print(sr)
print(wav.shape)

x = wav
x = torch.tensor(x).float() # casting the array into a PyTorch Tensor

spec_layer = STFT(n_fft=512, win_length=400, hop_length=160,
                              window='', freq_scale='linear', center=False, pad_mode='constant',
                              fmin=0, fmax=8000, sr=sr, output_format='Magnitude')
wav_spec = spec_layer(x) # Feed-forward your waveform to get the spectrogram
wav_spec = wav_spec[0].T
print(wav_spec.shape)


spec, rspec = fbank(wav, samplerate=16000,winlen=0.025,winstep=0.01,
          nfilt=40, nfft=512,lowfreq=0,highfreq=None,
          dither=0.0,remove_dc_offset=False, preemph=1.0, 
          wintype='hamming')
print(spec.shape)

print(wav_spec.numpy())
print(rspec)
# print(spec)

# spec, rspec = fbank(wav, samplerate=16000,winlen=0.032,winstep=0.01,
#           nfilt=40, nfft=512,lowfreq=0,highfreq=None,
#           dither=0.0,remove_dc_offset=False, preemph=1.0, 
#           wintype='hamming')
# print(rspec)

16000
(83792,)
sampling rate = 16000. Please make sure the sampling rate is correct in order toget a valid freq range
STFT kernels created, time used = 0.0153 seconds
torch.Size([521, 257])
(522, 257)
[[5.84560000e+04 2.55260664e+04 9.83611035e+03 ... 7.80710554e+00
  2.32206573e+01 1.90274487e+01]
 [1.35420000e+04 3.47535000e+04 1.51204707e+04 ... 1.69094101e+02
  1.80534729e+02 1.84179596e+02]
 [3.47560000e+04 2.83094609e+04 8.20204883e+03 ... 1.02080307e+02
  1.21321175e+02 1.08345497e+02]
 ...
 [9.36700000e+03 2.86213008e+04 1.41182402e+04 ... 1.19344498e+02
  1.25670158e+02 1.20691467e+02]
 [2.87510000e+04 2.04348242e+04 8.76390625e+03 ... 9.74485092e+01
  9.01831894e+01 9.84055099e+01]
 [4.45240000e+04 8.93593262e+03 4.39246826e+03 ... 6.16300154e+00
  8.94473553e+00 9.61348629e+00]]
[[5.89760000e+04 2.51006729e+04 8.59603890e+03 ... 2.02818313e+01
  2.40645984e+01 2.20000000e+01]
 [2.92660000e+04 2.72981079e+04 4.77242582e+03 ... 6.69265842e+01
  1.18775735e+02 1.62000000e+02]
 

In [129]:
print(np.hanning(10))
from scipy.signal import get_window
get_window('hann', 10, fftbins=True)

[0.         0.11697778 0.41317591 0.75       0.96984631 0.96984631
 0.75       0.41317591 0.11697778 0.        ]


array([0.       , 0.0954915, 0.3454915, 0.6545085, 0.9045085, 1.       ,
       0.9045085, 0.6545085, 0.3454915, 0.0954915])

In [153]:
f = frames(wav, samplerate=16000,winlen=0.025,winstep=0.01,
          nfilt=40, nfft=512,lowfreq=0,highfreq=None,
          dither=0.0,remove_dc_offset=False, preemph=1.0, 
          wintype='hamming')
print(f.shape)
print(f)

n_fft=512
freq_bins = n_fft//2+1
s = np.arange(0, n_fft, 1.)
wsin = np.empty((freq_bins,1,n_fft))
wcos = np.empty((freq_bins,1,n_fft))
for k in range(freq_bins): # Only half of the bins contain useful info
    wsin[k,0,:] = np.sin(2*np.pi*k*s/n_fft)
    wcos[k,0,:] = np.cos(2*np.pi*k*s/n_fft)


wsin = np.empty((n_fft,1,n_fft))
wcos = np.empty((n_fft,1,n_fft))
for k in range(n_fft): # Only half of the bins contain useful info
    wsin[k,0,:] = np.eye(n_fft, n_fft)[k]
    wcos[k,0,:] = np.eye(n_fft, n_fft)[k]
    
    
wsin = np.empty((400,1,n_fft))
wcos = np.empty((400,1,n_fft))
for k in range(400): # Only half of the bins contain useful info
    wsin[k,0,:] = np.eye(400, n_fft)[k]
    wcos[k,0,:] = np.eye(400, n_fft)[k]
    

    
x = torch.tensor(wav).float() # casting the array into a PyTorch Tensor
x = x[None, None, :]
print(x.size())
kernel_sin = torch.tensor(wsin, dtype=torch.float)
kernel_cos = torch.tensor(wcos, dtype=torch.float)
print(kernel_sin.size())

from torch.nn.functional import conv1d, conv2d, fold
spec_imag = conv1d(x, kernel_sin, stride=160)
spec_real = conv1d(x, kernel_cos, stride=160)  # Doing STFT by using conv1d

print(spec_imag.size())
print("conv frame", spec_imag[0].T)
# print(spec_imag[0].T[:, :400])

# remove redundant parts
# spec_real = spec_real[:, :freq_bins, :]
# spec_imag = spec_imag[:, :freq_bins, :]
# spec = spec_real.pow(2) + spec_imag.pow(2)
# spec = torch.sqrt(spec)
# print(spec)



s = np.arange(0, 512, 1.)
# s = s[::-1]
wsin = np.empty((freq_bins, 400))
wcos = np.empty((freq_bins, 400))
for k in range(freq_bins): # Only half of the bins contain useful info
    wsin[k,:] = np.sin(2*np.pi*k*s/n_fft)[:400]
    wcos[k,:] = np.cos(2*np.pi*k*s/n_fft)[:400]

spec_real = torch.mm(spec_imag[0].T, torch.tensor(wcos, dtype=torch.float).T)
spec_imag = torch.mm(spec_imag[0].T, torch.tensor(wsin, dtype=torch.float).T)


# remove redundant parts
spec = spec_real.pow(2) + spec_imag.pow(2)
spec = torch.sqrt(spec)

print('xx', spec.numpy())
print(spec.size())
print('yy', rspec[:521, :])
print('yy', rspec.shape)


x = spec.numpy()
y = rspec[:-1, :]
print(x)
print(y)
print(np.allclose(x, y))

(522, 400)
[[  43.   75.   69. ...   46.   46.   45.]
 [ 210.  215.  216. ...  -86.  -89.  -91.]
 [ 128.  128.  128. ... -154. -151. -151.]
 ...
 [ -60.  -61.  -61. ...  112.  109.  110.]
 [  20.   22.   24. ...   91.   87.   87.]
 [ 111.  107.  108. ...   -6.   -4.   -8.]]
torch.Size([1, 1, 83792])
torch.Size([400, 1, 512])
torch.Size([1, 400, 521])
conv frame tensor([[  43.,   75.,   69.,  ...,   46.,   46.,   45.],
        [ 210.,  215.,  216.,  ...,  -86.,  -89.,  -91.],
        [ 128.,  128.,  128.,  ..., -154., -151., -151.],
        ...,
        [-143., -141., -142.,  ...,   96.,  101.,  101.],
        [ -60.,  -61.,  -61.,  ...,  112.,  109.,  110.],
        [  20.,   22.,   24.,  ...,   91.,   87.,   87.]])
xx [[5.8976000e+04 2.5100676e+04 8.5960371e+03 ... 2.0281837e+01
  2.4064583e+01 2.2000000e+01]
 [2.9266000e+04 2.7298107e+04 4.7724253e+03 ... 6.6926659e+01
  1.1877571e+02 1.6200000e+02]
 [1.9630000e+04 2.8117480e+04 5.2880322e+03 ... 2.8501144e+01
  9.5781029e+01 1.42000

In [160]:
f = frames(wav, samplerate=16000,winlen=0.025,winstep=0.01,
          nfilt=40, nfft=512,lowfreq=0,highfreq=None,
          dither=0.0,remove_dc_offset=False, preemph=1.0, 
          wintype='hamming')

n_fft=512
freq_bins = n_fft//2+1
s = np.arange(0, n_fft, 1.)
wsin = np.empty((freq_bins,1,400))
wcos = np.empty((freq_bins,1,400)) #[Cout, Cin, kernel_size]
for k in range(freq_bins): # Only half of the bins contain useful info
    wsin[k,0,:] = np.sin(2*np.pi*k*s/n_fft)[:400]
    wcos[k,0,:] = np.cos(2*np.pi*k*s/n_fft)[:400]

    
x = torch.tensor(wav).float() # casting the array into a PyTorch Tensor
x = x[None, None, :] #[B, C, T]

kernel_sin = torch.tensor(wsin, dtype=torch.float)
kernel_cos = torch.tensor(wcos, dtype=torch.float)
print(kernel_sin.size())

from torch.nn.functional import conv1d, conv2d, fold
spec_imag = conv1d(x, kernel_sin, stride=160) #[1, Cout, T]
spec_real = conv1d(x, kernel_cos, stride=160)  # Doing STFT by using conv1d

# remove redundant parts
spec = spec_real.pow(2) + spec_imag.pow(2)
spec = torch.sqrt(spec)
print(spec)

x = spec[0].T.numpy()
y = rspec[:, :]
print(x)
print(y)
print(np.allclose(x, y))

torch.Size([257, 1, 400])
tensor([[[5.8976e+04, 2.9266e+04, 1.9630e+04,  ..., 1.6772e+04,
          3.8693e+04, 3.1020e+04],
         [2.5101e+04, 2.7298e+04, 2.8117e+04,  ..., 2.1323e+04,
          1.3598e+04, 1.5920e+04],
         [8.5960e+03, 4.7724e+03, 5.2880e+03,  ..., 4.0608e+02,
          6.7707e+03, 4.3020e+03],
         ...,
         [2.0282e+01, 6.6927e+01, 2.8501e+01,  ..., 2.6012e+01,
          6.1071e+01, 5.3685e+01],
         [2.4065e+01, 1.1878e+02, 9.5781e+01,  ..., 7.8405e+01,
          5.1310e+01, 6.3620e+01],
         [2.2000e+01, 1.6200e+02, 1.4200e+02,  ..., 9.0000e+01,
          3.5000e+01, 4.4000e+01]]])
[[5.8976000e+04 2.5100672e+04 8.5960391e+03 ... 2.0281828e+01
  2.4064537e+01 2.2000000e+01]
 [2.9266000e+04 2.7298107e+04 4.7724243e+03 ... 6.6926659e+01
  1.1877571e+02 1.6200000e+02]
 [1.9630000e+04 2.8117475e+04 5.2880312e+03 ... 2.8501148e+01
  9.5781006e+01 1.4200000e+02]
 ...
 [1.6772000e+04 2.1322793e+04 4.0607657e+02 ... 2.6011934e+01
  7.8405350e+01 9.

In [162]:
np.argmax(np.abs(x -y) / np.abs(y))

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


27241

In [165]:
y[np.unravel_index(27241, y.shape)]

0.0

In [166]:
x[np.unravel_index(27241, y.shape)]

4.2412265e-10

In [167]:
np.allclose(y, x)

False